In [56]:
import pandas as pd
import numpy as np

# Bestanden inlezen
mortgages_path = '/var/R/home/aa630798/Achmea Mortgagecashflows/Test_mortgages.csv'

# Lees de CSV-bestanden in
mortgages_df = pd.read_csv(mortgages_path, delimiter=',', encoding='ISO-8859-1')

# Datum parsing functie
def parse_dates(date_str):
    if isinstance(date_str, pd.Timestamp):
        return date_str
    for fmt in ('%m/%d/%Y', '%m-%d-%Y', '%Y-%m-%d', '%d-%m-%Y'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

# Pas de aangepaste datumparser toe
mortgages_df['Datum Ingang Leningdeel'] = mortgages_df['Datum Ingang Leningdeel'].apply(parse_dates)
mortgages_df['Datum Eind RVP'] = mortgages_df['Datum Eind RVP'].apply(parse_dates)
mortgages_df['Datum Eind Leningdeel'] = mortgages_df['Datum Eind Leningdeel'].apply(parse_dates)

# Spotdate instellen
spotdate = pd.Timestamp('2023-03-31')

print(mortgages_df)


    Leningdeel ID Hypotheek  Hoofdsom Oorspronkelijk  Hoofdsom Restant  \
0   2882182011            N                   185000         185000.00   
1   2882182012            N                   159000         103661.89   
2   2910630011            N                   266991         227003.76   
3   2910906012            N                    31321          31321.00   
4   2910906013            N                    37503          37503.00   
5   2910906011            N                   140074         119285.69   
6   2907709011            J                   140250         117613.09   
7   2908666011            J                   185130         159242.16   
8   2911836011            N                   145000          93417.74   
9   2911836012            N                    31800          31800.00   
10  2911836014            N                   118200         118200.00   
11  2911836013            N                    18980          16159.12   
12  2908331011            N           

In [57]:

# Unieke waardes bekijken
unique_rvps = mortgages_df['Looptijd Rentevastperiode 1'].unique()
unique_aflosvorm = mortgages_df['Aflosvorm'].unique()
print("Unieke Rentevaste Periodes:", unique_rvps)
print("Unieke Aflosvormen:", unique_aflosvorm)


Unieke Rentevaste Periodes: [ 12 120 360]
Unieke Aflosvormen: ['Aflossingsvrij' 'Lineair' 'Annuiteit' 'Levensverzekering']


In [58]:
num_steps = 360
num_morts = len(mortgages_df)

# Bepaal de resterende maanden vanaf de spotdate
mortgages_df['Months After Spotdate'] = (mortgages_df['Datum Eind RVP'].dt.to_period('M') - spotdate.to_period('M')).apply(lambda x: max(0, x.n))

# Pas n_periods aan naar de resterende maanden vanaf de spotdate
mortgages_df['n_periods_spotdate'] = mortgages_df[['Months After Spotdate', 'Looptijd Rentevastperiode 1']].min(axis=1)

# Bepaal de totale looptijd van de lening in maanden door het aantal dagen te delen door 30
mortgages_df['n_periods_total'] = (mortgages_df['Datum Eind Leningdeel'] - mortgages_df['Datum Ingang Leningdeel']).dt.days // 30

# Gebruik de resterende hoofdsom vanaf de spotdate
mortgages_df['Initial Principal'] = mortgages_df['Hoofdsom Restant']

print("Aantal hypotheken:", num_morts)


Aantal hypotheken: 17


In [59]:
# Consistente mapping tussen beide scripts
aflosvorm_mapping = {
    'Annuiteit': 'annuity',
    'Lineair': 'linear',
    'Aflossingsvrij': 'bullet',
    'Levensverzekering': 'bullet',  # Behandeld als aflossingsvrij
    'Spaarrekening': 'bullet',  # Behandeld als aflossingsvrij
    'Spaarverzekering': 'savings',
    'Beleggingshypotheek': 'investment',
    'Belegging': 'investment',
    'Leven': 'savings',
    'Beleggingsverzekering': 'investment'
}
print("Aflosvorm Mapping:", aflosvorm_mapping)

Aflosvorm Mapping: {'Annuiteit': 'annuity', 'Lineair': 'linear', 'Aflossingsvrij': 'bullet', 'Levensverzekering': 'bullet', 'Spaarrekening': 'bullet', 'Spaarverzekering': 'savings', 'Beleggingshypotheek': 'investment', 'Belegging': 'investment', 'Leven': 'savings', 'Beleggingsverzekering': 'investment'}


In [60]:
def annuity_cashflows(pv, rate, n_periods_rvp, n_periods_total):
    interest = np.zeros(n_periods_rvp)
    principal = np.zeros(n_periods_rvp)
    balance = np.zeros(n_periods_rvp)
    
    # Annuïteit betaling berekenen over de totale looptijd van de lening
    annuity_payment = pv * (rate / (1 - (1 + rate) ** -n_periods_total))
    remaining_balance = pv

    for t in range(n_periods_rvp):
        interest[t] = remaining_balance * rate
        principal[t] = annuity_payment - interest[t]
        remaining_balance -= principal[t]
        balance[t] = remaining_balance
    
    # Zorg ervoor dat de laatste betaling de resterende balans volledig aflost bij einde RVP
    if n_periods_rvp > 0:
        principal[-1] += balance[-2]
        balance[-1] = 0

    return interest, principal, balance




In [61]:
def linear_cashflows(pv, rate, n_periods):
    interest = np.zeros(n_periods)
    principal = np.full(n_periods, pv / n_periods)
    balance = np.zeros(n_periods)
    remaining_balance = pv

    for t in range(n_periods):
        interest[t] = remaining_balance * rate
        remaining_balance -= principal[t]
        balance[t] = remaining_balance
    
    # Zorg ervoor dat de laatste betaling de resterende balans volledig aflost
    if n_periods > 0:
        principal[-1] += balance[-2]
        balance[-1] = 0

    return interest, principal, balance



In [62]:
def interest_only_cashflows(pv, rate, n_periods):
    interest = np.full(n_periods, pv * rate)
    principal = np.zeros(n_periods)
    balance = np.full(n_periods, pv)
    
    # Voeg de volledige hoofdsom toe aan de laatste maand
    if n_periods > 0:
        principal[-1] = pv  # De laatste maand wordt de volledige aflossing van het restant
        balance[-1] = 0  # Na de laatste betaling is het saldo 0

    return interest, principal, balance


In [63]:
def savings_cashflows(pv, rate, n_periods):
    interest = np.zeros(n_periods)
    principal = np.zeros(n_periods)
    balance = np.zeros(n_periods)
    # Logic for savings cashflows can be added here
    return interest, principal, balance

def investment_cashflows(pv, rate, n_periods):
    interest = np.zeros(n_periods)
    principal = np.zeros(n_periods)
    balance = np.zeros(n_periods)
    # Logic for investment cashflows can be added here
    return interest, principal, balance


In [64]:
def calculate_cashflows(mortgages_df, spotdate):
    cashflows = []
    num_steps = 360

    for idx, row in mortgages_df.iterrows():
        aflosvorm = aflosvorm_mapping.get(row['Aflosvorm'], 'unknown')
        n_periods_rvp = row['n_periods_spotdate']
        n_periods_total = row['n_periods_total']

        if aflosvorm == 'annuity':
            interest, principal, balance = annuity_cashflows(row['Initial Principal'], row['Rente Nominaal'] / 12, n_periods_rvp, n_periods_total)
        elif aflosvorm == 'linear':
            interest, principal, balance = linear_cashflows(row['Initial Principal'], row['Rente Nominaal'] / 12, n_periods_rvp)
        elif aflosvorm == 'bullet' or aflosvorm == 'Aflossingsvrij':
            interest, principal, balance = interest_only_cashflows(row['Initial Principal'], row['Rente Nominaal'] / 12, n_periods_rvp)
        elif aflosvorm == 'savings' or aflosvorm == 'Levensverzekering':
            interest, principal, balance = savings_cashflows(row['Initial Principal'], row['Rente Nominaal'] / 12, n_periods_rvp)
        elif aflosvorm == 'investment':
            interest, principal, balance = investment_cashflows(row['Initial Principal'], row['Rente Nominaal'] / 12, n_periods_rvp)
        else:
            raise ValueError(f"Onbekende aflosvorm: {row['Aflosvorm']}")

        # Vul de resterende periodes met nullen (om te zorgen dat alle arrays 360 lang zijn)
        interest = np.pad(interest, (0, num_steps - len(interest)), 'constant')
        principal = np.pad(principal, (0, num_steps - len(principal)), 'constant')
        balance = np.pad(balance, (0, num_steps - len(balance)), 'constant')

        cashflows.append({
            'HypotheekID': row['Leningdeel'],
            'Aflosvorm': aflosvorm,
            'Interest': interest,
            'Principal': principal,
            'Balance': balance
        })

    return cashflows


In [65]:
# Cashflows berekenen
cashflows = calculate_cashflows(mortgages_df, spotdate)

# Print de cashflows voor alle hypotheken
for idx, cashflow in enumerate(cashflows):
    print(f"Hypotheek {idx + 1}:")
    print(f"Hypotheek ID: {cashflow['HypotheekID']}")
    print(f"Aflosvorm: {cashflow['Aflosvorm']}")
    print("Interest Cashflows:", cashflow['Interest'])
    print("Principal Cashflows:", cashflow['Principal'])
    print("Balance:", cashflow['Balance'])
    print("\n")


Hypotheek 1:
Hypotheek ID: 2882182011
Aflosvorm: bullet
Interest Cashflows: [172.66666667 172.66666667   0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.          